In [113]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
from torchsummary import summary

In [114]:
#Tensor->（multidimensional array）

#d input nodes k hidden layer
K = 10
d = 10
batch_size = 1

X = Variable(torch.rand(batch_size,d),requires_grad=True)

y = torch.tensor([[torch.sum(X**2)/d]])

In [115]:
X.shape

torch.Size([1, 10])

In [116]:
class NN1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        global z,z_relu
        z = []
        z_relu = []
        super(NN1, self).__init__()
        self.layer = nn.Sequential()
        self.layer.add_module("h_1", nn.Linear(input_size, hidden_size))
        
        z.append(self.layer(X))

        #self.layer.add_module("h_1_relu",nn.ReLU())
        z_relu.append(self.layer(X))
        for i in range(K-1):
             index = str(i+2)
             self.layer.add_module("h_"+index, nn.Linear(hidden_size, hidden_size))
             z.append(self.layer(X))
                
             self.layer.add_module("h_"+index+"_relu"+index,nn.ReLU())
             z_relu.append(self.layer(X))
                
        self.layer.add_module("h_last", nn.Linear(hidden_size, num_classes))
        z.append(self.layer(X))
        #self.layer.add_module("h_1ast_relu"+index,nn.ReLU())
        z_relu.append(self.layer(X))
        #print(len(self.layer))

    
    def forward(self, X):
        y = self.layer(X)
        return y
        

In [117]:
model = NN1(d,2*d+1,1)

In [118]:
y_pred = model(X)
loss = (y_pred - y)**2
loss.item()


0.3974741995334625

In [8]:
#summary(model,(1,10))

In [119]:
loss.backward()

--------------------

In [185]:
#for param in model.named_parameters():
#    print(param)

In [120]:
#save auto w,b
auto_w =[]
auto_b =[]
i = 0

for param in model.parameters():
    torch.set_printoptions(precision=5)
    if (i%2==0):
        auto_w.append(param.grad)
    if (i%2==1):
        auto_b.append(param.grad)
    i+=1

In [121]:
for i in range(len(auto_w)):
    auto_w[i]= np.round(np.array(auto_w[i]).astype(np.double),5)
    auto_b[i]= np.round(np.array(auto_b[i]).astype(np.double),5)

-----------------------

In [122]:
#手动grad share the same weight and bias with auto_forward
parameters = list(model.parameters())

In [123]:
#manual forward
def feedforward(X,parameters):
    global z_manual,z_relu_manual
    z_manual=[]
    z_relu_manual=[]
    z_manual.append(torch.from_numpy(np.dot(X.detach().numpy(),(parameters[0].detach().numpy().T))+parameters[1].detach().numpy()))
    z_relu_manual.append(z_manual[0])
    
    for i in range(0,10):
        z_manual.append(torch.from_numpy(np.dot(z_relu_manual[i],(parameters[(i+1)*2].detach().numpy().T))+parameters[(i+1)*2+1].detach().numpy()))
        z_relu_manual.append(ReLu(z_manual[i+1]))
    z_relu_manual[-1]=z_manual[-1]
    return z_relu_manual[-1]
y_pred = feedforward(X,parameters)

In [125]:
#求对z的的倒数
# dL/dv5
d_z = []
d_w = []
grad_y_pred = 2.0 * (y_pred - y) #[1,1]
grad_z_y = grad_y_pred.detach().numpy()#*ReLu_d(y_pred)#dL/dz5
grad_w_y = grad_z_y*z_relu_manual[-2]#.detach().numpy()
d_z.append(grad_z_y)
d_w.append(grad_w_y)
#from the last hidden layer
for i in range (9):#i=1,2,3,4,5.... p:-4,-6,-8
    grad_z_i = np.dot(d_z[i],ReLu_d(z_manual[-i-2])*parameters[-(i+1)*2].detach().numpy())
    d_z.append(grad_z_i)
    grad_w_i = np.dot(grad_z_i.T,z_relu_manual[-3-i])#.detach().numpy()) #-4,-5,-6
    d_w.append(grad_w_i)
    
#first layer
grad_z_1 = np.dot(d_z[9],ReLu_d(z_manual[-11])*parameters[-20].detach().numpy())
d_z.append(grad_z_1)
grad_w_1 = np.dot(grad_z_1.T,X.detach().numpy())
d_w.append(grad_w_1)


-------------

In [89]:
#激活函数的倒数
def ReLu_d(x):
    return np.where(x < 0, 0, 1)
def ReLu(x):
    return np.where(x < 0, 0, x)

In [126]:
for i in range(len(d_w)):
    d_w[i]= np.round(np.array(d_w[i]).astype(np.double),5)
    d_z[i]= np.round(np.array(d_z[i]).astype(np.double),5)

In [127]:
d_w.reverse()

In [128]:
d_z.reverse()

In [129]:
d_w

[array([[ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 4.e-05,  4.e-05,  2.e-05,  3.e-05,  3.e-05,  3.e-05,  2.e-05,
          2.e-05,  4.e-05,  3.e-05],
        [-3.e-05, -3.e-05, -2.e-05, -2.e-05, -2.e-05, -3.e-05, -1.e-05,
         -1.e-05, -3.e-05, -2.e-05],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [-2.e-05, -

In [130]:
auto_w

[array([[ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [-3.e-05, -3.e-05, -2.e-05, -2.e-05, -3.e-05, -3.e-05, -1.e-05,
         -2.e-05, -3.e-05, -2.e-05],
        [ 1.e-05,  1.e-05,  0.e+00,  0.e+00,  1.e-05,  1.e-05,  0.e+00,
          0.e+00,  1.e-05,  0.e+00],
        [ 0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,  0.e+00,
          0.e+00,  0.e+00,  0.e+00],
        [ 3.e-05,  2.e-05,  1.e-05,  2.e-05,  2.e-05,  2.e-05,  1.e-05,
          1.e-05,  2.e-05,  2.e-05],
        [-5.e-05, -5.e-05, -3.e-05, -3.e-05, -4.e-05, -4.e-05, -2.e-05,
         -2.e-05, -5.e-05, -3.e-05],
        [ 4.e-05,  4.e-05,  2.e-05,  3.e-05,  3.e-05,  3.e-05,  2.e-05,
          2.e-05,  4.e-05,  3.e-05],
        [-3.e-05, -3.e-05, -2.e-05, -2.e-05, -2.e-05, -3.e-05, -1.e-05,
         -1.e-05, -3.e-05, -2.e-05],
        [-1.e-05, -1.e-05, -1.e-05, -1.e-05, -1.e-05, -1.e-05, -0.e+00,
         -0.e+00, -1.e-05, -1.e-05],
        [-2.e-05, -

In [131]:
with open('torch_autograd.dat', 'w') as f:
    f.write('autograd_w' + '\n')
    i = 1
    for w in auto_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('autograd_b' + '\n')
    i = 1
    for b in auto_b:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    

In [132]:
with open('my_autograd.dat', 'w') as f:
    f.write('my_w' + '\n')
    i = 1
    for w in d_w:
        f.write('weight_' +str(i)+ '\n')
        f.write(str(w)+'\n')
        i+=1
    f.write('my_b' + '\n')
    i = 1
    for b in d_z:
        f.write('bias_' +str(i)+ '\n')
        f.write(str(b)+'\r\n')
        i+=1
    

In [136]:
for i in range(len(auto_w)):
    print(auto_w[i].all()==d_w[i].all())

True
True
True
True
True
True
True
True
True
True
True


In [134]:
for i in range(len(auto_b)):
    print(auto_b[i].all()==d_z[i].all())

True
True
True
True
True
True
True
True
True
True
True
